In [1]:
import ee
ee.Initialize(project='tmospp')

In [2]:
%load_ext autoreload
%autoreload 2

from rat.core.run_sarea import run_sarea

/tiger1/pdas47/tmsosPP/.env/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)
/tiger1/pdas47/tmsosPP/.env/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for JRC/GSW1_3/GlobalSurfaceWater! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_3_GlobalSurfaceWater

  warnings.warn(warning, category=DeprecationWarning)


In [3]:
from pathlib import Path
import geopandas as gpd
import hvplot.pandas
import holoviews as hv

hv.extension('bokeh')

In [7]:
RESERVOIR = '1078'

RESULTS_DIR = Path(f'../results/')
DATA_DIR = Path(f'../data/')

In [22]:
val_pts = gpd.read_file(Path('../data/validation-locations/2023-24-insitu-pts.geojson'))
val_polys = gpd.read_file(Path('../data/validation-locations/2023-24-insitu-poly.geojson'))

selected_reservoirs = val_polys['tmsos_id'].tolist()  # select all 100 reservoirs
res_names = val_polys[['tmsos_id', 'name']].set_index('tmsos_id').to_dict()['name'] # dictionary that can be queried to get reservoir name

RESERVOIR_NAME = res_names[RESERVOIR]

val_res_pt = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]
val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

global_map = (
    val_res_pt.hvplot(
        geo=True, tiles='OSM'
    ) * val_res_pt[val_res_pt['tmsos_id'] == RESERVOIR].hvplot(
        geo=True, color='red', size=100, 
    )
).opts(
    title=f"Locations of validation reservoirs. {RESERVOIR_NAME}, highlighted in red"
)

global_map

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]
   .Polygons.II :Polygons   [Longitude,Latitude]

In [23]:
(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR].hvplot(
    geo=True, tiles='OSM', shared_axes=False
)).opts(title=f"{RESERVOIR_NAME}")

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [24]:
BUFFER_M = 800 # m

utm_crs = val_polys[val_polys['tmsos_id'] == RESERVOIR].estimate_utm_crs()
print(f"UTM area of use: \n{utm_crs.area_of_use}")

buffered_roi_utm = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR].to_crs(utm_crs).buffer(BUFFER_M).to_crs('epsg:4326')
buffered_roi = buffered_roi_utm.to_crs('epsg:4326')

UTM area of use: 
- name: Between 120°W and 114°W, northern hemisphere between equator and 84°N, onshore and offshore. Canada - Alberta; British Columbia (BC); Northwest Territories (NWT); Nunavut. Mexico. United States (USA).
- bounds: (-120.0, 0.0, -114.0, 84.0)


## Run TMS-OS over the selected reservoirs

In [26]:
# tmsos requires the start and end date, reservoir shapefile, a dictionary of columns names for unique_identifier and nominal area, and a datadirectory
import pandas as pd


start_date = '2019-01-01'
end_date = '2024-10-31'
reservoir_shpfile = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]
shpfile_column_dict = {
    'unique_identifier': 'tmsos_id',
    'area_column': 'AREA_SKM'
}
datadir = Path('../data/tmsos/')
datadir.mkdir(exist_ok=True)

run_sarea(
    start_date = start_date, 
    end_date = end_date, 
    datadir = datadir,
    reservoirs_shpfile = reservoir_shpfile, 
    shpfile_column_dict = shpfile_column_dict, 
)



 +++ PROCESSING RESERVOIR: 1078 - Lake Mead, US (1/1) +++




Existing file found - Last observation (10 day lag): 2024-10-31 00:00:00
No. of days passed since: 0
No new observation expected. Quitting early
Existing file found - Last observation (32 day lag): 2024-10-19 00:00:00
No. of days passed since: 12
No new observation expected. Quitting early
Existing file found - Last observation (32 day lag): 2024-10-27 00:00:00
No. of days passed since: 4
No new observation expected. Quitting early
Existing file found - Last observation (24 day lag): 2024-10-19 13:35:33
No. of days passed since: 11
No new observation expected. Quitting early
Duplicated labels, deleting
Duplicated labels, deleting


Now we need to change the format of the data to the one expected by (04-test.ipynb)[04-test.ipynb].

Expected format:
| time    | [algorithm name] area [unit] | tmsos_id
| -------- | ------- | ------ |
| %%%  | ###    | ^^^ |
| %%% | ###     | ^^^ |
| %%%    | ###    | ^^^ |

In [ ]:
from pathlib import Path
tmsos_dir = Path('../data/tmsos')
version = 'tmsos'

tmsospp_fmt_dfs = []
for reservoir in selected_reservoirs:
    tmsos_fp = tmsos_dir / (reservoir + '.csv')
    tmsos_area_df = pd.read_csv(tmsos_fp, parse_dates=['date'])
    
    tmsospp_fmt_df = tmsos_area_df.rename({
        'date': 'time',
        'area': 'tmsos area [km2]',
    }, axis=1)
    tmsospp_fmt_df = tmsospp_fmt_df[['time', 'tmsos area [km2]']]
    tmsospp_fmt_df['tmsos_id'] = reservoir

    save_fp = Path(f'../data/area/tmsos/{version}/') / f"{reservoir}.csv"
    save_fp.parent.mkdir(exist_ok=True, parents=True)

    tmsospp_fmt_df.to_csv(save_fp, index=False)